In [0]:
from pyspark.sql.functions import col
import numpy as np 
import random
import numpy as np
from pyspark.sql import Row
from sklearn import neighbors
import math

In [0]:
# Load the joined airlines and weather observations in from Parquet
blob_container = "w261" # The name of your container created in https://portal.azure.com
storage_account = "bdougall" # The name of your Storage account created in https://portal.azure.com
blob_url = f"wasbs://{blob_container}@{storage_account}.blob.core.windows.net"
mount_path = "/mnt/mids-w261"

In [0]:
train_and_val =  spark.read.parquet(f"{blob_url}/pipeline_trainval").cache()
display(train_and_val)

YEAR,MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,TAIL_NUM,ORIGIN,DEST,CRS_ARR_TIME,CRS_ELAPSED_TIME,DISTANCE,utc_scheduled_departure,utc_2hrs_before,wind_obs_type_origin,wind_obs_speed_origin,ceil_height_origin,vis_dist_origin,air_temp_origin,dew_pt_temp_origin,wind_obs_type_destination,wind_obs_speed_destination,ceil_height_destination,vis_dist_destination,air_temp_destination,dew_pt_temp_destination,HOUR,OUTCOME,rolling_perc_delayed,origin_dest_perc_24to2,origin_perc_24to2,last_flight_delayed,amount_last_delay,two_ago_delayed,three_ago_delayed,arrival_time_2_next_flight,last_flight_late_arrival,two_ago_late_arrival,three_ago_late_arrival,features,label
2018,1,3,2018-01-03,G4,218NV,PIE,SBN,1017,156.0,973.0,2018-01-03T12:41:00.000+0000,2018-01-03T10:41:00.000+0000,N,62,244,9656,61,56,N,72,3048,16093,-139,-183,7,1,50.0,0.0,5.555555555555555,1,152.0,1,1,46560,1,1,1,"Map(vectorType -> dense, length -> 30, values -> List(17.0, 0.0, 0.0, 2.0, 1.0, 1.0, 2.0, 1.0, 1.0, 2018.0, 1.0, 3.0, 1017.0, 156.0, 973.0, 62.0, 244.0, 9656.0, 61.0, 56.0, 72.0, 3048.0, 16093.0, -139.0, -183.0, 7.0, 50.0, 0.0, 5.555555555555555, 152.0))",1.0
2018,1,3,2018-01-03,G4,218NV,SBN,PIE,1343,156.0,973.0,2018-01-03T16:07:00.000+0000,2018-01-03T14:07:00.000+0000,N,41,2134,8047,-128,-139,N,82,183,3219,83,78,11,1,50.0,0.0,53.84615384615385,1,36.0,1,1,3000,-1,1,1,"Map(vectorType -> dense, length -> 30, values -> List(17.0, 0.0, 0.0, 2.0, 1.0, 1.0, 0.0, 1.0, 1.0, 2018.0, 1.0, 3.0, 1343.0, 156.0, 973.0, 41.0, 2134.0, 8047.0, -128.0, -139.0, 82.0, 183.0, 3219.0, 83.0, 78.0, 11.0, 50.0, 0.0, 53.84615384615385, 36.0))",1.0
2018,1,3,2018-01-03,G4,218NV,PIE,FWA,1939,147.0,912.0,2018-01-03T22:12:00.000+0000,2018-01-03T20:12:00.000+0000,N,88,22000,16093,122,61,N,57,610,4023,-83,-117,17,0,0.0,0.0,27.77777777777778,1,109.0,1,1,12540,1,1,1,"Map(vectorType -> dense, length -> 30, values -> List(17.0, 0.0, 0.0, 2.0, 1.0, 1.0, 2.0, 1.0, 1.0, 2018.0, 1.0, 3.0, 1939.0, 147.0, 912.0, 88.0, 22000.0, 16093.0, 122.0, 61.0, 57.0, 610.0, 4023.0, -83.0, -117.0, 17.0, 0.0, 0.0, 27.77777777777778, 109.0))",0.0
2018,1,3,2018-01-03,G4,218NV,FWA,PIE,2248,144.0,912.0,2018-01-04T01:24:00.000+0000,2018-01-03T23:24:00.000+0000,N,57,1524,9656,-83,-111,N,67,22000,16093,89,44,20,1,100.0,0.0,27.77777777777778,0,1.0,1,1,2700,-1,1,1,"Map(vectorType -> dense, length -> 30, values -> List(17.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 2018.0, 1.0, 3.0, 2248.0, 144.0, 912.0, 57.0, 1524.0, 9656.0, -83.0, -111.0, 67.0, 22000.0, 16093.0, 89.0, 44.0, 20.0, 100.0, 0.0, 27.77777777777778, 1.0))",1.0
2018,1,4,2018-01-04,G4,218NV,PIE,TOL,1007,149.0,945.0,2018-01-04T12:38:00.000+0000,2018-01-04T10:38:00.000+0000,N,26,22000,16093,56,-17,N,46,762,16093,-122,-144,7,0,0.0,0.0,38.46153846153847,1,66.0,0,1,31800,1,1,1,"Map(vectorType -> dense, length -> 30, values -> List(17.0, 0.0, 0.0, 2.0, 0.0, 1.0, 2.0, 1.0, 1.0, 2018.0, 1.0, 4.0, 1007.0, 149.0, 945.0, 26.0, 22000.0, 16093.0, 56.0, -17.0, 46.0, 762.0, 16093.0, -122.0, -144.0, 7.0, 0.0, 0.0, 38.46153846153847, 66.0))",0.0
2018,1,4,2018-01-04,G4,218NV,TOL,PIE,1321,149.0,945.0,2018-01-04T15:52:00.000+0000,2018-01-04T13:52:00.000+0000,N,41,22000,16093,-144,-183,N,31,1372,16093,56,6,10,1,0.0,0.0,16.666666666666664,0,-3.0,1,0,2700,-1,1,1,"Map(vectorType -> dense, length -> 30, values -> List(17.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 2018.0, 1.0, 4.0, 1321.0, 149.0, 945.0, 41.0, 22000.0, 16093.0, -144.0, -183.0, 31.0, 1372.0, 16093.0, 56.0, 6.0, 10.0, 0.0, 0.0, 16.666666666666664, -3.0))",1.0
2018,1,5,2018-01-05,G4,218NV,PIE,CVG,1022,131.0,776.0,2018-01-05T13:11:00.000+0000,2018-01-05T11:11:00.000+0000,N,51,22000,16093,44,-44,N,15,1219,16093,-172,-194,8,0,0.0,0.0,36.36363636363637,1,141.0,1,1,42780,1,1,1,"Map(vectorType -> dense, length -> 30, values -> List(17.0, 0.0, 0.0, 2.0, 1.0, 1.0, 2.0, 1.0, 1.0, 2018.0, 1.0, 5.0, 1022.0, 131.0, 776.0, 51.0, 22000.0, 16093.0, 44.0, -44.0, 15.0, 1219.0, 16093.0, -172.0, -194.0, 8.0, 0.0, 0.0, 36.36363636363637, 141.0))",0.0
2018,

In [0]:
def smote_rdd(df):
    """Implement SMOTE using sci-kit learn nearest neighbor, RDD operations, and downsampling as necessary"""
    
    # Filter the dataframe for minority and majority class observations
    major_obs = df.filter(col('label') == 0)
    minor_obs = df.filter(col('label') == 1)
    
    # Get the count of major class observations
    major_counts = major_obs.count()
    minor_counts = minor_obs.count()
    
    # There cannot be more nearest neighbors than there are examples of the minority class
    try_k = int(major_counts/minor_counts)
    if try_k > minor_counts:
        k = minor_counts
    else:
        k = try_k
    
    # Select the features field from the dataframe of minority class observations and convert to an RDD
    featureMin = minor_obs.select('features').rdd
    # Extract the features and create a numpy array from them
    feature_val = featureMin.map(lambda x: x[0]).collect()
    feature = np.asarray(feature_val)
    
    # Create a nearest neighbors classifier and fit to the minority class features
    nearest_nbrs = neighbors.NearestNeighbors(n_neighbors=k).fit(feature)
    # Get the k-nearest neighbors for each row
    row_nbrs =  nearest_nbrs.kneighbors(feature)
    row_nbrs = row_nbrs[1]
    
    # Create a list to hold the new rows
    newRows = []
    
    for idx in range(len(feature_val)):
        for i in row_nbrs[idx]:
            newRec = feature_val[idx] + ((feature_val[idx] - feature_val[i])*random.random())
            newRows.insert(0,(newRec))
    # Create a new RDD from the new rows
    newData_rdd = sc.parallelize(newRows)
    # Create a dataframe from the RDD of the new rows
    newData_rdd_new = newData_rdd.map(lambda x: Row(features = x, label = 1))
    new_data = newData_rdd_new.toDF()
    # Join the synthetic features back to the minor observations
    new_data_minor = minor_obs.select('features', 'label').unionAll(new_data)
    
    # Find the count of records in the new dataframe
    minor_counts = new_data_minor.count()
    
    # Find the ratio of major class to synthetic + original minor class observations
    try:
        ratio = int(major_counts/minor_counts)
        
        # Select only the necessary columns from the major dataframe
        major_df = major_obs.select('features', 'label')
    
        # If the ratio is approximately 1, join the new dataframe of synthetic + original minority class observations to the major class observations
        if ratio <= 1:      
            smoted_df = major_df.unionAll(new_data_minor)
        else:
            # Downsample the majority class
            reduced_majority = major_df.sample(False, 1/ratio)
            # Create a new dataset, with fewer majority class observations
            smoted_df = reduced_majority.unionAll(new_data_minor)
    
        return smoted_df
    except ZeroDivisionError:
        return (f'There were no minority class observations in the sample')

In [0]:
# Select 50 records from the training and validation set and checkpoint them for testing with SMOTE
trial_50 = train_and_val.limit(50).cache()

# Display the class counts before and after 
row_nbrs = smote_rdd(trial_50).cache()
display(trial_50.groupBy('label').count())
display(row_nbrs.groupBy('label').count())

label,count
0.0,41
1.0,9


label,count
1.0,45
0.0,41


In [0]:
# Create smote dataframes the two training years
smoted_2015 = smote_rdd(train_and_val.filter(col('YEAR') == 2015))